In [16]:
import requests
import os
import warnings
from tqdm import tqdm
import numpy as np
import zipfile
import rioxarray
import xarray as xr
from ipyleaflet import Map, basemaps, LayersControl, WidgetControl
from rasterio.warp import Resampling
import matplotlib.pyplot as plt
import zarr
import s3fs
from dotenv import load_dotenv
import os
import ipywidgets as widgets
from ipywidgets import interact
from traitlets import link
from localtileserver import get_leaflet_tile_layer, TileClient
from tempfile import NamedTemporaryFile
import warnings

from zarr_visualization.zarr_visualization import ZarrVisualization

warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
zarr_viz = ZarrVisualization(bucket_name='tobfer')

@interact
def select_data(
    variable_select=zarr_viz.variable_select):

    list_of_variables_subdata = zarr_viz.fs.ls(f"{zarr_viz.bucket_name}/{variable_select}")
    list_of_variables_subdata = [item.split('/')[-1] for item in list_of_variables_subdata]
    subdata_select = widgets.Dropdown(
        options=list_of_variables_subdata,
        value=list_of_variables_subdata[0],  
        description='Files:'
    )
    subdata_select.observe(zarr_viz.update_data_path, names='value')
    display(subdata_select, zarr_viz.printed_data_path)
    display(zarr_viz.open_button)

    display(zarr_viz.time_counter_container)  
    display(zarr_viz.lat_min_container)  
    display(zarr_viz.lat_max_container)
    display(zarr_viz.lon_min_container)
    display(zarr_viz.lon_max_container)
    display(zarr_viz.sel_button)
    display(zarr_viz.data_information)
    display(zarr_viz.show_data)
    display(zarr_viz.loading_container)
    zarr_viz.open_button.on_click(lambda b: zarr_viz.process_zarr_file(subdata_select))
    
zarr_viz.m

interactive(children=(Dropdown(description='Variable:', options=('T1d', 'T1m', 'T1y', 'T5d', 'U1d', 'U1m', 'U1…

Map(center=[40, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [11]:
# ds = xr.open_zarr('https://noc-msm-o.s3-ext.jc.rl.ac.uk/tobfer/V1m/vo.zarr')